<img src="https://industrial.uniandes.edu.co/sites/default/files/imagenes/uniandeslogo.png" alt="Universidad de los Andes" style="float: right; width: 300px; height: auto;">

# Cleaning Ministerio de Defensa Nacional de Colombia data

Autor: Juan Diego Heredia Niño 

Email: jd.heredian@uniandes.edu.co

Date: Oct 2025

In [1]:
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import yaml  # To read YAML configuration files
from pathlib import Path  # For cross-platform file path handling

In [ ]:
# Load directory paths from configuration file
with open('paths.yml', 'r') as file:
    paths = yaml.safe_load(file)  # Read and parse YAML file

# Create Path objects for each directory
raw = Path(paths['data']['raw'])  # Directory with raw data
temp = Path(paths['data']['temp'])  # Directory with temporary processed data
processed = Path(paths['data']['processed'])  # Directory with final processed data

## Homicides

In [ ]:
# Read Excel file with intentional homicide data
df_homicides = pd.read_excel(raw / 'mindef' / 'top5' / 'HOMICIDIO INTENCIONAL.xlsx')

# Convert date to monthly period (first day of month) then to date format
df_homicides['FECHA_HECHO'] = df_homicides['FECHA_HECHO'].dt.to_period('M').dt.to_timestamp().dt.date

# Standardize municipality code: convert to 5-digit string (zero-padded)
df_homicides['COD_MUNI'] = df_homicides['COD_MUNI'].astype(str).str.zfill(5)

# Group by date and municipality, summing total victims
df_homicides = df_homicides.groupby(['FECHA_HECHO', 'COD_MUNI'])[['VICTIMAS']].sum().reset_index()

# Rename columns to English for standardization
df_homicides.rename(columns={'FECHA_HECHO': 'date', 'COD_MUNI': 'mun_code', 'VICTIMAS': 'qty'}, inplace=True)

# Assign crime code (1 = Homicide)
df_homicides['crime_code'] = '01'

# Save result in Parquet format (more efficient than CSV)
df_homicides.to_parquet(temp / 'mindef' / 'top5' /  'homicides.parquet', index=False)

In [4]:
# Sanity checks for homicides
print("HOMICIDES - Data Quality Checks:")
print(f"  Total records: {len(df_homicides):,}")
print(f"  Date range: {df_homicides['date'].min()} to {df_homicides['date'].max()}")
print(f"  Unique municipalities: {df_homicides['mun_code'].nunique()}")
print(f"  Total victims: {df_homicides['qty'].sum():,}")
print(f"  Null values: {df_homicides.isnull().sum().sum()}")
print(f"  Negative quantities: {(df_homicides['qty'] < 0).sum()}")
print(f"  Duplicates: {df_homicides.duplicated(subset=['date', 'mun_code']).sum()}")
print()

HOMICIDES - Data Quality Checks:
  Total records: 87,828
  Date range: 2003-01-01 to 2025-09-01
  Unique municipalities: 1108
  Total victims: 331,667
  Null values: 0
  Negative quantities: 0
  Duplicates: 0



In [ ]:
# KIDNAPPINGS
# Read Excel file with kidnapping data
df_kidnappings = pd.read_excel(raw / 'mindef' / 'top5' / 'SECUESTRO.xlsx')

# Convert date to monthly period (first day of month) then to date format
df_kidnappings['FECHA_HECHO'] = df_kidnappings['FECHA_HECHO'].dt.to_period('M').dt.to_timestamp().dt.date

# Standardize municipality code: convert to 5-digit string (zero-padded)
df_kidnappings['COD_MUNI'] = df_kidnappings['COD_MUNI'].astype(str).str.zfill(5)

# Group by date and municipality, summing the number of events
df_kidnappings = df_kidnappings.groupby(['FECHA_HECHO', 'COD_MUNI'])[['CANTIDAD']].sum().reset_index()

# Rename columns to English for standardization
df_kidnappings.rename(columns={'FECHA_HECHO': 'date', 'COD_MUNI': 'mun_code', 'CANTIDAD': 'qty'}, inplace=True)

# Assign crime code (2 = Kidnapping)
df_kidnappings['crime_code'] = '02'

# Save result in Parquet format
df_kidnappings.to_parquet(temp / 'mindef' / 'top5' /  'kidnappings.parquet', index=False)

In [6]:
# Sanity checks for kidnappings
print("KIDNAPPINGS - Data Quality Checks:")
print(f"  Total records: {len(df_kidnappings):,}")
print(f"  Date range: {df_kidnappings['date'].min()} to {df_kidnappings['date'].max()}")
print(f"  Unique municipalities: {df_kidnappings['mun_code'].nunique()}")
print(f"  Total events: {df_kidnappings['qty'].sum():,}")
print(f"  Null values: {df_kidnappings.isnull().sum().sum()}")
print(f"  Negative quantities: {(df_kidnappings['qty'] < 0).sum()}")
print(f"  Duplicates: {df_kidnappings.duplicated(subset=['date', 'mun_code']).sum()}")
print()

KIDNAPPINGS - Data Quality Checks:
  Total records: 6,145
  Date range: 2003-01-01 to 2025-09-01
  Unique municipalities: 838
  Total events: 10,254
  Null values: 0
  Negative quantities: 0
  Duplicates: 0



In [ ]:
# TERRORISM
# Read Excel file with terrorism data
df_terrorsim = pd.read_excel(raw / 'mindef' / 'top5' / 'TERRORISMO.xlsx')

# Convert date to monthly period (first day of month) then to date format
df_terrorsim['FECHA_HECHO'] = df_terrorsim['FECHA_HECHO'].dt.to_period('M').dt.to_timestamp().dt.date

# Standardize municipality code: convert to 5-digit string (zero-padded)
df_terrorsim['COD_MUNI'] = df_terrorsim['COD_MUNI'].astype(str).str.zfill(5)

# Group by date and municipality, summing the number of events
df_terrorsim = df_terrorsim.groupby(['FECHA_HECHO', 'COD_MUNI'])[['CANTIDAD']].sum().reset_index()

# Rename columns to English for standardization
df_terrorsim.rename(columns={'FECHA_HECHO': 'date', 'COD_MUNI': 'mun_code', 'CANTIDAD': 'qty'}, inplace=True)

# Assign crime code (3 = Terrorism)
df_terrorsim['crime_code'] = '03'

# Save result in Parquet format
df_terrorsim.to_parquet(temp / 'mindef' / 'top5' /  'terrorism.parquet', index=False)

In [8]:
# Sanity checks for terrorism
print("TERRORISM - Data Quality Checks:")
print(f"  Total records: {len(df_terrorsim):,}")
print(f"  Date range: {df_terrorsim['date'].min()} to {df_terrorsim['date'].max()}")
print(f"  Unique municipalities: {df_terrorsim['mun_code'].nunique()}")
print(f"  Total events: {df_terrorsim['qty'].sum():,}")
print(f"  Null values: {df_terrorsim.isnull().sum().sum()}")
print(f"  Negative quantities: {(df_terrorsim['qty'] < 0).sum()}")
print(f"  Duplicates: {df_terrorsim.duplicated(subset=['date', 'mun_code']).sum()}")
print()

TERRORISM - Data Quality Checks:
  Total records: 9,229
  Date range: 2003-01-01 to 2025-09-01
  Unique municipalities: 762
  Total events: 14,313
  Null values: 0
  Negative quantities: 0
  Duplicates: 0



In [ ]:
# EXTORTION
# Read Excel file with extortion data
df_extortion = pd.read_excel(raw / 'mindef' / 'top5' / 'EXTORSIÓN.xlsx')

# Convert date to monthly period (first day of month) then to date format
df_extortion['FECHA_HECHO'] = df_extortion['FECHA_HECHO'].dt.to_period('M').dt.to_timestamp().dt.date

# Standardize municipality code: convert to 5-digit string (zero-padded)
df_extortion['COD_MUNI'] = df_extortion['COD_MUNI'].astype(str).str.zfill(5)

# Group by date and municipality, summing the number of events
df_extortion = df_extortion.groupby(['FECHA_HECHO', 'COD_MUNI'])[['CANTIDAD']].sum().reset_index()

# Rename columns to English for standardization
df_extortion.rename(columns={'FECHA_HECHO': 'date', 'COD_MUNI': 'mun_code', 'CANTIDAD': 'qty'}, inplace=True)

# Assign crime code (4 = Extortion)
df_extortion['crime_code'] = '04'

# Save result in Parquet format
df_extortion.to_parquet(temp / 'mindef' / 'top5' /  'extortion.parquet', index=False)

In [10]:
# Sanity checks for extortion
print("EXTORTION - Data Quality Checks:")
print(f"  Total records: {len(df_extortion):,}")
print(f"  Date range: {df_extortion['date'].min()} to {df_extortion['date'].max()}")
print(f"  Unique municipalities: {df_extortion['mun_code'].nunique()}")
print(f"  Total events: {df_extortion['qty'].sum():,}")
print(f"  Null values: {df_extortion.isnull().sum().sum()}")
print(f"  Negative quantities: {(df_extortion['qty'] < 0).sum()}")
print(f"  Duplicates: {df_extortion.duplicated(subset=['date', 'mun_code']).sum()}")
print()

EXTORTION - Data Quality Checks:
  Total records: 37,761
  Date range: 2003-01-01 to 2025-09-01
  Unique municipalities: 1086
  Total events: 117,877
  Null values: 0
  Negative quantities: 0
  Duplicates: 0



In [ ]:
# MASSACRES
# Read Excel file with massacre data
df_massacres = pd.read_excel(raw / 'mindef' / 'top5' / 'MASACRES.xlsx')

# Convert date to monthly period (first day of month) then to date format
df_massacres['FECHA_HECHO'] = df_massacres['FECHA_HECHO'].dt.to_period('M').dt.to_timestamp().dt.date

# Standardize municipality code: convert to 5-digit string (zero-padded)
df_massacres['COD_MUNI'] = df_massacres['COD_MUNI'].astype(str).str.zfill(5)

# Group by date and municipality, summing total victims
df_massacres = df_massacres.groupby(['FECHA_HECHO', 'COD_MUNI'])[['VICTIMAS']].sum().reset_index()

# Rename columns to English for standardization
df_massacres.rename(columns={'FECHA_HECHO': 'date', 'COD_MUNI': 'mun_code', 'VICTIMAS': 'qty'}, inplace=True)

# Assign crime code (5 = Massacres)
df_massacres['crime_code'] = '05'

# Save result in Parquet format
df_massacres.to_parquet(temp / 'mindef' / 'top5' /  'massacres.parquet', index=False)

In [12]:
# Sanity checks for massacres
print("MASSACRES - Data Quality Checks:")
print(f"  Total records: {len(df_massacres):,}")
print(f"  Date range: {df_massacres['date'].min()} to {df_massacres['date'].max()}")
print(f"  Unique municipalities: {df_massacres['mun_code'].nunique()}")
print(f"  Total victims: {df_massacres['qty'].sum():,}")
print(f"  Null values: {df_massacres.isnull().sum().sum()}")
print(f"  Negative quantities: {(df_massacres['qty'] < 0).sum()}")
print(f"  Duplicates: {df_massacres.duplicated(subset=['date', 'mun_code']).sum()}")
print()

MASSACRES - Data Quality Checks:
  Total records: 342
  Date range: 2022-01-01 to 2025-09-01
  Unique municipalities: 190
  Total victims: 1,225
  Null values: 0
  Negative quantities: 0
  Duplicates: 0



## Standard Cleaning Process

This notebook processes 5 types of crimes from the Colombian Ministry of Defense:

1. **Intentional Homicide** (crime_code = 1)
2. **Kidnapping** (crime_code = 2)
3. **Terrorism** (crime_code = 3)
4. **Extortion** (crime_code = 4)
5. **Massacres** (crime_code = 5)

### Process steps for each file:

1. **Reading**: Load Excel file from `data/raw/mindef/top5/`
2. **Date normalization**: Convert `FECHA_HECHO` to monthly period (format: YYYY-MM-01)
3. **Code standardization**: Convert `COD_MUNI` to 5-digit zero-padded string
4. **Aggregation**: Group by date and municipality, summing victims/event count
5. **Renaming**: Standardize column names to English (`date`, `mun_code`, `qty`)
6. **Encoding**: Add `crime_code` column to identify crime type
7. **Export**: Save in Parquet format to `data/temp/mindef/top5/`

### Output structure:
Each processed file has the same structure:
- `date`: Event date (first day of month)
- `mun_code`: Municipality code (5 digits)
- `qty`: Number of victims or events
- `crime_code`: Crime type identifier (1-5)